# ═══════════════════════════════════════════════════════════════════════════
# 🔬 THE PATTERN HUNTER'S LAB
# Aortic Arch Transformer: Tracing Vertebrate Circulatory Evolution
# ═══════════════════════════════════════════════════════════════════════════
# 
# 📖 Companion to: "Pattern Hunter's Guide" - Unit 3: Circulatory System
# 🎯 Learning Goals:
#    - Visualize embryonic aortic arch patterns
#    - Track evolutionary transformations across vertebrate classes
#    - Identify homologous structures in adult circulation
#    - Understand developmental constraints on evolution
# ═══════════════════════════════════════════════════════════════════════════

## 📦 SETUP: Install and Import Libraries

In [ ]:
!pip install -q plotly kaleido ipywidgets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
import os
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set visual style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded successfully!")
print("🎨 Ready to explore aortic arch evolution!")

## 📚 PART 1: THE PATTERN HUNTER'S BRIEFING

In [ ]:
display(Markdown("""
## 🎯 The Mystery: Why Six Arches?

All vertebrate embryos start with **six pairs of aortic arches** connecting the ventral 
aorta to the dorsal aortae. But adult vertebrates show wildly different patterns:

- **Fish**: Keep most arches (feeding gills)
- **Amphibians**: Modified pattern (dual life)  
- **Reptiles**: Partial reduction (three vessels from heart)
- **Birds**: Right arch dominant (supports flight metabolism)
- **Mammals**: Left arch dominant (supports endothermy)

### 🔍 The Pattern Hunter's Questions:
1. Why do ALL embryos start with the same 6-arch pattern?
2. What determines which arches persist vs. disappear?
3. Can we predict arch fate from lifestyle/physiology?
4. What evolutionary constraints does this reveal?

---
**🧬 Evolutionary Insight**: Embryonic recapitulation doesn't mean "ontogeny recapitulates 
phylogeny" but rather that development is constrained by ancestral patterns.
"""))

## 🗂️ PART 2: THE DATA - Aortic Arch Fate Maps

In [ ]:
# Create comprehensive dataset of aortic arch fates
arch_data = pd.DataFrame({
    'Arch': [1, 2, 3, 4, 5, 6] * 5,
    'Species_Group': ['Fish']*6 + ['Amphibian']*6 + ['Reptile']*6 + ['Bird']*6 + ['Mammal']*6,
    'Embryonic_Present': [True]*30,  # All present in embryo
    'Adult_Fate': [
        # Fish (retain most for gills)
        'Jaw vessels', 'Hyoid vessels', 'Gill arch I', 'Gill arch II', 'Gill arch III', 'Pulmonary',
        # Amphibian
        'Lost', 'Lost', 'Carotid arch', 'Systemic arch (both)', 'Lost', 'Pulmo-cutaneous',
        # Reptile (generalized pattern)
        'Lost', 'Lost', 'Carotid arch', 'Systemic arches (L&R)', 'Lost', 'Pulmonary arch',
        # Bird
        'Lost', 'Lost', 'Carotid arch', 'Right systemic only', 'Lost', 'Pulmonary arch',
        # Mammal
        'Lost', 'Lost', 'Carotid arch', 'Left systemic (aorta)', 'Lost', 'Pulmonary + ductus'
    ],
    'Functional_Role': [
        # Fish
        'Head circulation', 'Hyoid/spiracle', 'Gas exchange', 'Gas exchange', 'Gas exchange', 'Gas exchange',
        # Amphibian
        'None', 'None', 'Brain/head', 'Body circulation', 'None', 'Lung + skin',
        # Reptile
        'None', 'None', 'Brain/head', 'Body circulation', 'None', 'Lung',
        # Bird
        'None', 'None', 'Brain/head', 'Body (high pressure)', 'None', 'Lung',
        # Mammal
        'None', 'None', 'Brain/head', 'Body (systemic)', 'None', 'Lung (fetal bypass)'
    ],
    'Persistence': [
        # Fish
        'Full', 'Full', 'Full', 'Full', 'Full', 'Full',
        # Amphibian
        'Absent', 'Absent', 'Partial', 'Full', 'Absent', 'Full',
        # Reptile
        'Absent', 'Absent', 'Partial', 'Full', 'Absent', 'Full',
        # Bird
        'Absent', 'Absent', 'Partial', 'Right only', 'Absent', 'Full',
        # Mammal
        'Absent', 'Absent', 'Partial', 'Left only', 'Absent', 'Full'
    ]
})

# Phylogenetic relationship data
phylo_order = ['Fish', 'Amphibian', 'Reptile', 'Bird', 'Mammal']
arch_data['Species_Group'] = pd.Categorical(arch_data['Species_Group'], 
                                            categories=phylo_order, 
                                            ordered=True)

display(Markdown("### 📊 Aortic Arch Fate Database"))
display(arch_data.head(12))

print(f"\n📈 Dataset contains {len(arch_data)} arch observations across {arch_data['Species_Group'].nunique()} vertebrate groups")

## 🎨 PART 3: VISUALIZATION - The Arch Transformer

### Interactive Visualization 1: Embryonic vs. Adult Patterns
Explore how the universal 6-arch embryonic pattern transforms into diverse adult forms.

In [ ]:
def create_arch_diagram(species_group):
    """Create schematic diagram of aortic arch pattern"""
    
    data = arch_data[arch_data['Species_Group'] == species_group]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Embryonic Stage (6 arches)', f'Adult {species_group}'),
        specs=[[{'type': 'scatter'}, {'type': 'scatter'}]]
    )
    
    # Embryonic pattern (all 6 arches present)
    for arch in range(1, 7):
        # Ventral aorta to arch
        fig.add_trace(go.Scatter(
            x=[0, 0.3, 0.5], 
            y=[0, arch*0.5, arch*0.5],
            mode='lines',
            line=dict(color='lightblue', width=8),
            showlegend=False,
            hoverinfo='text',
            text=f'Arch {arch} (Embryonic)'
        ), row=1, col=1)
        
        # Arch to dorsal aorta
        fig.add_trace(go.Scatter(
            x=[0.5, 0.7, 1], 
            y=[arch*0.5, arch*0.5, 0],
            mode='lines',
            line=dict(color='lightblue', width=8),
            showlegend=False,
            hoverinfo='text',
            text=f'Arch {arch} (Embryonic)'
        ), row=1, col=1)
    
    # Adult pattern (selective persistence)
    for _, row in data.iterrows():
        arch = row['Arch']
        persistence = row['Persistence']
        
        if persistence != 'Absent':
            color = 'red' if persistence == 'Full' else 'orange'
            width = 10 if persistence == 'Full' else 6
            
            # Adjust for asymmetry in birds/mammals
            x_offset = 0
            if species_group == 'Bird' and arch == 4:
                x_offset = 0.1  # Right side
            elif species_group == 'Mammal' and arch == 4:
                x_offset = -0.1  # Left side
            
            # Ventral to arch
            fig.add_trace(go.Scatter(
                x=[1.5, 1.8+x_offset, 2+x_offset], 
                y=[0, arch*0.5, arch*0.5],
                mode='lines',
                line=dict(color=color, width=width),
                showlegend=False,
                hoverinfo='text',
                text=f'Arch {arch}: {row["Adult_Fate"]}<br>{row["Functional_Role"]}'
            ), row=1, col=2)
            
            # Arch to dorsal
            fig.add_trace(go.Scatter(
                x=[2+x_offset, 2.2+x_offset, 2.5], 
                y=[arch*0.5, arch*0.5, 0],
                mode='lines',
                line=dict(color=color, width=width),
                showlegend=False,
                hoverinfo='text',
                text=f'Arch {arch}: {row["Adult_Fate"]}<br>{row["Functional_Role"]}'
            ), row=1, col=2)
    
    # Add heart and vessel labels
    fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers+text',
                            marker=dict(size=30, color='pink'),
                            text=['Heart'], textposition='bottom center',
                            showlegend=False), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=[1.5], y=[0], mode='markers+text',
                            marker=dict(size=30, color='pink'),
                            text=['Heart'], textposition='bottom center',
                            showlegend=False), row=1, col=2)
    
    fig.update_xaxes(showgrid=False, showticklabels=False, zeroline=False)
    fig.update_yaxes(showgrid=False, showticklabels=False, zeroline=False)
    
    fig.update_layout(
        height=500,
        title_text=f"Aortic Arch Transformation: {species_group}",
        title_font_size=16,
        hovermode='closest'
    )
    
    return fig

print("✅ Visualization function created!")
print("📊 Run the next cell to see the interactive diagram")

In [ ]:
# Interactive widget with manual update
@interact
def show_arch_transformation(species=['Fish', 'Amphibian', 'Reptile', 'Bird', 'Mammal']):
    """Display interactive arch transformation diagram"""
    fig = create_arch_diagram(species)
    fig.show()
    
    # Display key information
    data = arch_data[arch_data['Species_Group'] == species]
    retained = data[data['Persistence'] != 'Absent']
    
    print(f"\n🔍 {species} Pattern Summary:")
    print(f"   Arches retained: {len(retained)}/6")
    print(f"\n   Active arches:")
    for _, row in retained.iterrows():
        print(f"      • Arch {row['Arch']}: {row['Adult_Fate']} ({row['Persistence']})")

## 📊 PART 4: COMPARATIVE ANALYSIS - Pattern Recognition

### Pattern Detection: Which Arches Persist?
Let's quantify the evolutionary pattern across vertebrates.

In [ ]:
# Create persistence matrix
persistence_matrix = arch_data.pivot_table(
    index='Arch',
    columns='Species_Group',
    values='Persistence',
    aggfunc='first',
    observed=True
)

# Convert to numeric for visualization
persistence_numeric = persistence_matrix.replace({
    'Full': 2,
    'Partial': 1,
    'Right only': 1,
    'Left only': 1,
    'Absent': 0
}).astype(int)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(persistence_numeric, 
            annot=persistence_matrix.values,
            fmt='',
            cmap='RdYlGn',
            cbar_kws={'label': 'Persistence Level'},
            linewidths=2,
            linecolor='white',
            ax=ax)

plt.title('Aortic Arch Persistence Map Across Vertebrates', fontsize=14, fontweight='bold')
plt.xlabel('Vertebrate Group (Phylogenetic Order)', fontsize=12)
plt.ylabel('Aortic Arch Number', fontsize=12)
plt.tight_layout()
plt.show()

# Statistical summary
display(Markdown("### 📈 Key Pattern Statistics"))

persistence_summary = arch_data.groupby('Species_Group', observed=True)['Persistence'].value_counts().unstack(fill_value=0)
display(persistence_summary)

# Calculate arch retention rate
retention_rate = arch_data[arch_data['Persistence'] != 'Absent'].groupby('Species_Group', observed=True).size() / 6 * 100

print("\n🎯 Arch Retention Rate by Group:")
for group, rate in retention_rate.items():
    print(f"   {group}: {rate:.1f}% of embryonic arches retained in adult")

## 🧬 PART 5: EVOLUTIONARY TRENDS - The Pattern Emerges

### Evolutionary Trend Analysis
How does arch simplification relate to vertebrate evolution?

In [ ]:
# Create trend visualization
fig_trend = go.Figure()

for arch in range(1, 7):
    arch_subset = arch_data[arch_data['Arch'] == arch].copy()
    arch_subset['Numeric_Persistence'] = arch_subset['Persistence'].map({
        'Full': 2,
        'Partial': 1,
        'Right only': 1,
        'Left only': 1,
        'Absent': 0
    })
    
    fig_trend.add_trace(go.Scatter(
        x=arch_subset['Species_Group'],
        y=arch_subset['Numeric_Persistence'],
        mode='lines+markers',
        name=f'Arch {arch}',
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig_trend.update_layout(
    title='Aortic Arch Persistence Across Vertebrate Evolution',
    xaxis_title='Vertebrate Group (Phylogenetic Sequence)',
    yaxis_title='Persistence Level',
    yaxis=dict(
        tickmode='array',
        tickvals=[0, 1, 2],
        ticktext=['Absent', 'Partial/Asymmetric', 'Full']
    ),
    height=500,
    hovermode='x unified',
    legend_title='Aortic Arch'
)

fig_trend.show()

## 🎮 PART 6: INTERACTIVE CHALLENGE - Pattern Hunter's Test

### CHALLENGE: Predict the Pattern!

You've discovered a **new fossil vertebrate** with these characteristics:
- Fully terrestrial (no aquatic life stage)
- Endothermic metabolism  
- Four-chambered heart
- High-pressure systemic circulation

**Question**: Which aortic arch pattern would you predict for the adult form?

Use the sliders below to indicate which arches you think persist:

In [ ]:
# Create interactive prediction tool
arch_sliders = {}
for i in range(1, 7):
    arch_sliders[f'arch_{i}'] = widgets.SelectionSlider(
        options=['Absent', 'Partial', 'Full'],
        value='Absent',
        description=f'Arch {i}:',
        style={'description_width': '80px'}
    )

predict_button = widgets.Button(
    description='🔍 Check Prediction',
    button_style='success',
    tooltip='Compare your prediction with actual patterns'
)

output_area = widgets.Output()

def check_prediction(b):
    with output_area:
        output_area.clear_output()
        
        # Get user predictions
        predictions = {i: arch_sliders[f'arch_{i}'].value for i in range(1, 7)}
        
        # Expected patterns (bird or mammal-like)
        bird_pattern = {1: 'Absent', 2: 'Absent', 3: 'Partial', 4: 'Full', 5: 'Absent', 6: 'Full'}
        mammal_pattern = {1: 'Absent', 2: 'Absent', 3: 'Partial', 4: 'Full', 5: 'Absent', 6: 'Full'}
        
        # Calculate match score
        bird_score = sum(predictions[i] == bird_pattern[i] for i in range(1, 7))
        mammal_score = sum(predictions[i] == mammal_pattern[i] for i in range(1, 7))
        
        print("=" * 70)
        print("🎯 PATTERN HUNTER'S VERDICT")
        print("=" * 70)
        print(f"\n📊 Your prediction matches:")
        print(f"   🦅 Bird pattern: {bird_score}/6 arches ({bird_score/6*100:.0f}%)")
        print(f"   🦝 Mammal pattern: {mammal_score}/6 arches ({mammal_score/6*100:.0f}%)")
        
        if bird_score >= 5 or mammal_score >= 5:
            print("\n✅ EXCELLENT! You've identified the endothermic pattern!")
            print("\n🔬 Key insights:")
            print("   • Arches 1, 2, 5 are ALWAYS lost in amniotes")
            print("   • Arch 3 persists partially (carotid arteries)")
            print("   • Arch 4 provides main systemic circulation")
            print("   • Arch 6 specialized for pulmonary circulation")
            print("\n🧬 Evolutionary constraint: Complete separation of pulmonary")
            print("   and systemic circulation is REQUIRED for endothermy!")
        else:
            print("\n💡 Not quite! Here's what endothermic vertebrates show:")
            print("\n   Expected pattern:")
            print("   • Arches 1, 2, 5: Lost (no function in air-breathers)")
            print("   • Arch 3: Partial (carotid arteries to head)")
            print("   • Arch 4: Full (main systemic trunk - ONE side dominant)")
            print("   • Arch 6: Full (pulmonary circulation)")
            print("\n   💭 Think about: Why must pulmonary and systemic be separate?")

predict_button.on_click(check_prediction)

display(widgets.VBox([
    widgets.HTML("<h3>🎯 Your Prediction:</h3>"),
    *arch_sliders.values(),
    predict_button,
    output_area
]))

## 🎓 PART 7: DEEPER DIVE - Functional Constraints

### Advanced Analysis: Why These Patterns?

#### Key Evolutionary Principles

##### 1. **Developmental Constraint**
All vertebrates START with 6 arches because:
- Ancestral pharyngeal arch development (from gill-bearing ancestors)
- Modifying existing structures is easier than creating new ones
- Neural crest cell migration patterns are conserved

##### 2. **Functional Requirement**
Pattern is driven by **circulatory needs**:

In [ ]:
# Create functional requirements table
functional_data = pd.DataFrame({
    'Vertebrate Group': ['Fish', 'Amphibian', 'Reptile', 'Bird', 'Mammal'],
    'Metabolism': ['Ectothermic', 'Ectothermic', 'Mostly Ectothermic', 'Endothermic', 'Endothermic'],
    'Blood Pressure Need': ['Low', 'Low-Medium', 'Medium', 'High', 'High'],
    'Pulmonary Separation': ['None (gills)', 'Partial', 'Partial/Complete', 'Complete', 'Complete'],
    'Arches Retained': [6, 4, 4, 3, 3],
    'Key Adaptation': ['Gas exchange', 'Dual life', 'Terrestrial', 'Flight', 'Endothermy']
})

display(functional_data)

# Visualize relationship
fig_func, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Arch retention vs metabolic rate
metabolic_order = ['Ectothermic', 'Mostly Ectothermic', 'Endothermic']
functional_data['Metabolic_Numeric'] = functional_data['Metabolism'].map({
    'Ectothermic': 1,
    'Mostly Ectothermic': 2,
    'Endothermic': 3
})

axes[0].scatter(functional_data['Metabolic_Numeric'], 
                functional_data['Arches Retained'],
                s=200, alpha=0.6, c=range(5), cmap='viridis')

for i, row in functional_data.iterrows():
    axes[0].annotate(row['Vertebrate Group'], 
                     (row['Metabolic_Numeric'], row['Arches Retained']),
                     xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[0].set_xlabel('Metabolic Rate', fontsize=12)
axes[0].set_ylabel('Number of Arches Retained', fontsize=12)
axes[0].set_title('Metabolic Rate vs. Arch Simplification', fontsize=12, fontweight='bold')
axes[0].set_xticks([1, 2, 3])
axes[0].set_xticklabels(['Ectothermic', 'Mostly\nEctothermic', 'Endothermic'])
axes[0].grid(True, alpha=0.3)

# Plot 2: Separation trend
separation_map = {'None (gills)': 0, 'Partial': 1, 'Partial/Complete': 1.5, 'Complete': 2}
functional_data['Separation_Numeric'] = functional_data['Pulmonary Separation'].map(separation_map)

axes[1].plot(range(5), functional_data['Separation_Numeric'], 
             'o-', markersize=12, linewidth=3, color='darkred', alpha=0.7)

for i, row in functional_data.iterrows():
    axes[1].annotate(row['Vertebrate Group'], 
                     (i, row['Separation_Numeric']),
                     xytext=(0, -15), textcoords='offset points', 
                     fontsize=9, ha='center')

axes[1].set_xlabel('Evolutionary Sequence', fontsize=12)
axes[1].set_ylabel('Pulmonary-Systemic Separation', fontsize=12)
axes[1].set_title('Evolution of Circulatory Separation', fontsize=12, fontweight='bold')
axes[1].set_xticks(range(5))
axes[1].set_xticklabels(functional_data['Vertebrate Group'], rotation=45)
axes[1].set_yticks([0, 1, 1.5, 2])
axes[1].set_yticklabels(['None', 'Partial', 'Variable', 'Complete'])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🏆 PART 8: PATTERN HUNTER'S CONCLUSIONS

### THE PATTERNS REVEALED

#### Universal Principles Discovered:

##### 1. **Conservation of Development** 
The 6-arch embryonic pattern is CONSERVED across all vertebrates because:
- Evolution modifies existing developmental programs
- Complete redesign would require coordinating many changes
- "Good enough" solutions are retained

##### 2. **Adaptive Modification**
Adult patterns reflect FUNCTIONAL NEEDS:
- **Fish**: Retain all arches (gill circulation)
- **Amphibians**: Modified for dual aquatic-terrestrial life
- **Amniotes**: Arches 1, 2, 5 consistently lost (no gill function)
- **Endotherms**: Complete pulmonary-systemic separation (metabolic demand)

##### 3. **Phylogenetic Signal**
Arch IV asymmetry is a **phylogenetic marker**:
- 🦅 **Birds**: RIGHT arch IV persists
- 🦝 **Mammals**: LEFT arch IV persists
- This reflects independent evolution of endothermy!

##### 4. **Constraint + Selection**
Evolution works within constraints:
- Must start with 6-arch pattern (developmental constraint)
- Selective pressure shapes which persist (functional requirement)
- Result: Limited number of viable adult patterns

---

### Questions for Further Investigation:

1. **Comparative**: Why do birds keep RIGHT arch IV while mammals keep LEFT?
2. **Developmental**: What molecular signals determine which arches regress?
3. **Functional**: Could a tetrapod function with a different arch pattern?
4. **Evolutionary**: What transitional forms show intermediate patterns?

---

### 📚 Return to Main Text
Now that you've explored the aortic arch pattern, return to **Chapter 3.2** 
to see how this integrates with heart evolution and systemic circulation.

### 🧪 Next Lab
Ready for more pattern hunting? Try:
- **Lab 3.3**: Heart Chamber Evolution Simulator
- **Lab 3.4**: Blood Pressure Requirements Calculator
- **Lab 3.5**: Comparative Embryology Explorer

## 💾 PART 9: COMPREHENSIVE EXPORT SYSTEM

### Export Your Complete Analysis
Save all data, figures, and findings with one click!

In [ ]:
# COMPREHENSIVE EXPORT SYSTEM
from google.colab import files
from google.colab import drive
import shutil
from datetime import datetime

# Create timestamp for unique filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Export configuration
export_config = {
    'save_csv': widgets.Checkbox(value=True, description='CSV Data'),
    'save_summary': widgets.Checkbox(value=True, description='Summary Report'),
    'save_heatmap': widgets.Checkbox(value=True, description='Heatmap Figure'),
    'save_diagrams': widgets.Checkbox(value=True, description='All Species Diagrams'),
    'save_trends': widgets.Checkbox(value=True, description='Trend Charts'),
    'save_functional': widgets.Checkbox(value=True, description='Functional Analysis'),
}

# Export destination
export_destination = widgets.RadioButtons(
    options=['Download to Computer', 'Save to Google Drive', 'Both'],
    value='Download to Computer',
    description='Destination:',
    style={'description_width': 'initial'}
)

export_button = widgets.Button(
    description='📥 EXPORT EVERYTHING',
    button_style='success',
    tooltip='Export all selected files',
    layout=widgets.Layout(width='300px', height='50px')
)

export_output = widgets.Output()

def export_everything(b):
    with export_output:
        export_output.clear_output()
        
        print("\n" + "=" * 70)
        print("📦 COMPREHENSIVE EXPORT INITIATED")
        print("=" * 70 + "\n")
        
        exported_files = []
        
        # 1. Export CSV Data
        if export_config['save_csv'].value:
            filename = f'aortic_arch_data_{timestamp}.csv'
            arch_data.to_csv(filename, index=False)
            exported_files.append(filename)
            print(f"✅ CSV Data: {filename}")
        
        # 2. Export Summary Report
        if export_config['save_summary'].value:
            filename = f'arch_analysis_summary_{timestamp}.txt'
            with open(filename, 'w') as f:
                f.write("="*70 + "\n")
                f.write("AORTIC ARCH PATTERN ANALYSIS\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("="*70 + "\n\n")
                
                f.write("RETENTION RATES:\n")
                f.write("-" * 50 + "\n")
                for group, rate in retention_rate.items():
                    f.write(f"  {group:15s}: {rate:5.1f}% of embryonic arches retained\n")
                
                f.write("\n" + "="*70 + "\n")
                f.write("KEY FINDINGS:\n")
                f.write("="*70 + "\n")
                f.write("\n1. UNIVERSAL EMBRYONIC PATTERN\n")
                f.write("   All vertebrate embryos begin with 6 paired aortic arches\n")
                f.write("   This reflects ancestral gill-bearing condition\n")
                
                f.write("\n2. PHYLOGENETIC PATTERNS\n")
                f.write("   Fish: Retain all 6 arches for gill circulation\n")
                f.write("   Amphibians: 4 arches retained (dual aquatic-terrestrial life)\n")
                f.write("   Reptiles: 4 arches retained (terrestrial adaptation)\n")
                f.write("   Birds: 3 arches retained (RIGHT arch IV dominant)\n")
                f.write("   Mammals: 3 arches retained (LEFT arch IV dominant)\n")
                
                f.write("\n3. EVOLUTIONARY CONSTRAINTS\n")
                f.write("   Arches 1, 2, 5 consistently lost in amniotes\n")
                f.write("   Arch 3 persists as carotid arteries\n")
                f.write("   Arch 4 provides systemic circulation\n")
                f.write("   Arch 6 specialized for pulmonary function\n")
                
                f.write("\n4. FUNCTIONAL SIGNIFICANCE\n")
                f.write("   Endotherms require complete pulmonary-systemic separation\n")
                f.write("   Bird/Mammal arch IV asymmetry reflects independent evolution\n")
                f.write("   Pattern constrained by developmental mechanisms\n")
                
                f.write("\n" + "="*70 + "\n")
                f.write("END OF REPORT\n")
                f.write("="*70 + "\n")
            
            exported_files.append(filename)
            print(f"✅ Summary Report: {filename}")
        
        # 3. Export Heatmap
        if export_config['save_heatmap'].value:
            filename = f'arch_heatmap_{timestamp}.png'
            plt.figure(figsize=(10, 6))
            sns.heatmap(persistence_numeric, annot=persistence_matrix.values, fmt='',
                       cmap='RdYlGn', linewidths=2, linecolor='white',
                       cbar_kws={'label': 'Persistence Level'})
            plt.title('Aortic Arch Persistence Map Across Vertebrates', fontsize=14, fontweight='bold')
            plt.xlabel('Vertebrate Group (Phylogenetic Order)', fontsize=12)
            plt.ylabel('Aortic Arch Number', fontsize=12)
            plt.tight_layout()
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            plt.close()
            exported_files.append(filename)
            print(f"✅ Heatmap Figure: {filename}")
        
        # 4. Export All Species Diagrams
        if export_config['save_diagrams'].value:
            for species in ['Fish', 'Amphibian', 'Reptile', 'Bird', 'Mammal']:
                # HTML version (interactive)
                html_filename = f'arch_diagram_{species.lower()}_{timestamp}.html'
                fig = create_arch_diagram(species)
                fig.write_html(html_filename)
                exported_files.append(html_filename)
                
                # PNG version (static)
                png_filename = f'arch_diagram_{species.lower()}_{timestamp}.png'
                fig.write_image(png_filename, width=1200, height=600)
                exported_files.append(png_filename)
            
            print(f"✅ Species Diagrams: 5 species × 2 formats = 10 files")
        
        # 5. Export Trend Chart
        if export_config['save_trends'].value:
            html_filename = f'arch_trends_{timestamp}.html'
            png_filename = f'arch_trends_{timestamp}.png'
            fig_trend.write_html(html_filename)
            fig_trend.write_image(png_filename, width=1200, height=600)
            exported_files.append(html_filename)
            exported_files.append(png_filename)
            print(f"✅ Trend Charts: {html_filename}, {png_filename}")
        
        # 6. Export Functional Analysis
        if export_config['save_functional'].value:
            csv_filename = f'functional_analysis_{timestamp}.csv'
            png_filename = f'functional_analysis_{timestamp}.png'
            
            functional_data.to_csv(csv_filename, index=False)
            exported_files.append(csv_filename)
            
            # Recreate and save the functional analysis figure
            fig_func.savefig(png_filename, dpi=300, bbox_inches='tight')
            exported_files.append(png_filename)
            print(f"✅ Functional Analysis: {csv_filename}, {png_filename}")
        
        print(f"\n📊 Total files created: {len(exported_files)}")
        print("\n" + "=" * 70)
        
        # Handle export destination
        destination = export_destination.value
        
        if destination in ['Download to Computer', 'Both']:
            print("\n⬇️  DOWNLOADING FILES TO YOUR COMPUTER...\n")
            for filename in exported_files:
                if os.path.exists(filename):
                    files.download(filename)
                    print(f"   ⬇️  {filename}")
            print("\n✅ All files downloaded! Check your Downloads folder.")
        
        if destination in ['Save to Google Drive', 'Both']:
            print("\n☁️  SAVING TO GOOGLE DRIVE...\n")
            try:
                # Mount Google Drive
                drive.mount('/content/drive', force_remount=True)
                
                # Create directory structure
                base_dir = '/content/drive/MyDrive/Comparative_Anatomy_Labs'
                session_dir = f'{base_dir}/Session_{timestamp}'
                
                os.makedirs(session_dir, exist_ok=True)
                os.makedirs(f'{session_dir}/data', exist_ok=True)
                os.makedirs(f'{session_dir}/figures', exist_ok=True)
                os.makedirs(f'{session_dir}/reports', exist_ok=True)
                
                # Copy files to appropriate folders
                for filename in exported_files:
                    if filename.endswith('.csv'):
                        shutil.copy(filename, f'{session_dir}/data/{filename}')
                    elif filename.endswith('.txt'):
                        shutil.copy(filename, f'{session_dir}/reports/{filename}')
                    else:
                        shutil.copy(filename, f'{session_dir}/figures/{filename}')
                
                print(f"✅ Files saved to Google Drive!")
                print(f"📁 Location: MyDrive/Comparative_Anatomy_Labs/Session_{timestamp}/")
                print(f"   ├── data/ ({sum(1 for f in exported_files if f.endswith('.csv'))} files)")
                print(f"   ├── figures/ ({sum(1 for f in exported_files if not f.endswith(('.csv', '.txt')))} files)")
                print(f"   └── reports/ ({sum(1 for f in exported_files if f.endswith('.txt'))} files)")
                
            except Exception as e:
                print(f"⚠️  Google Drive save failed: {e}")
                print("💡 Tip: Make sure you authorize Drive access when prompted")
        
        print("\n" + "=" * 70)
        print("🎉 EXPORT COMPLETE!")
        print("=" * 70)
        print("\n💡 Next Steps:")
        print("   • Use CSV files for further analysis in Excel/R/Python")
        print("   • Include figures in your reports and presentations")
        print("   • Share HTML diagrams for interactive exploration")
        print("   • Reference the summary report for key findings")
        print("\n")

export_button.on_click(export_everything)

# Display export interface
display(widgets.VBox([
    widgets.HTML("<h2>📥 Comprehensive Export System</h2>"),
    widgets.HTML("<p>Select what you want to export:</p>"),
    widgets.VBox(list(export_config.values())),
    widgets.HTML("<br><p><b>Export Destination:</b></p>"),
    export_destination,
    widgets.HTML("<br>"),
    export_button,
    export_output
]))

### 📚 File Organization Guide

#### When saved to Google Drive, your files will be organized as:

```
MyDrive/
└── Comparative_Anatomy_Labs/
    └── Session_YYYYMMDD_HHMMSS/
        ├── data/
        │   ├── aortic_arch_data_*.csv
        │   └── functional_analysis_*.csv
        ├── figures/
        │   ├── arch_heatmap_*.png
        │   ├── arch_diagram_fish_*.html
        │   ├── arch_diagram_fish_*.png
        │   └── ... (all other diagrams)
        └── reports/
            └── arch_analysis_summary_*.txt
```

#### File Types Explained:

- **CSV files**: Open in Excel, Google Sheets, or import into R/Python
- **PNG files**: High-resolution images for reports and presentations
- **HTML files**: Interactive diagrams you can open in any web browser
- **TXT files**: Plain text summaries for quick reference

#### Usage Tips:

1. **For Reports**: Use PNG files in Word/PowerPoint
2. **For Analysis**: Import CSV files into your favorite tool
3. **For Sharing**: Send HTML files for interactive exploration
4. **For Study**: Read TXT summaries for key concepts

---
## 🎓 Congratulations, Pattern Hunter!

You've successfully:
- ✅ Traced embryonic to adult arch transformations
- ✅ Identified phylogenetic patterns in circulatory evolution  
- ✅ Connected structure to metabolic function
- ✅ Understood developmental constraints on evolution
- ✅ Exported comprehensive analysis for future use

### 🌟 Pattern Hunter Skills Earned:
- **Homology Recognition**: Identifying conserved embryonic patterns
- **Functional Morphology**: Linking structure to physiological demands
- **Phylogenetic Analysis**: Using anatomy to infer evolutionary relationships
- **Predictive Biology**: Using patterns to make evolutionary predictions
- **Data Management**: Organizing and exporting scientific analyses

---

### 📖 Citation
*The Pattern Hunter's Lab: Aortic Arch Transformer*  
Companion to "The Pattern Hunter's Guide to Comparative Anatomy"  
Interactive Colab Notebook - Unit 3: Circulatory System

---

**🔗 Share this notebook**: File → Save a copy → Share link  
**💬 Discuss patterns**: [Link to course discussion forum]  
**🐛 Report issues**: [Link to feedback form]

---

*Happy Pattern Hunting! 🔍🧬*